# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../WeatherPy/output_Data/cities.csv', encoding="utf-8")
cities_df.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Bluff,100,NZ,1651962575,98,-46.6000,168.3333,57.79,6.02
1,Alofi,96,NU,1651962575,89,-19.0595,-169.9187,78.69,3.44
2,Kirakira,81,SB,1651962575,80,-10.4544,161.9205,82.76,10.51
3,Manjacaze,0,MZ,1651962533,93,-24.7117,33.8828,64.36,6.06
4,Kavieng,59,PG,1651962387,71,-2.5744,150.7967,84.11,17.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Fill Humidity values and convert to float
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
humidity = cities_df["Humidity"].tolist()
markers = gmaps.marker_layer(locations,info_box_content=[f"Humidity: {humidity}" for humidity in humidity])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Preferred City - Max Temp Greater than 60 and less than 80, Humidity less than 75, Cloudiness less than 30%
pref_cities = cities_df.loc[
    (cities_df["Max Temp"] > 60) &
    (cities_df["Max Temp"] < 80) &
    (cities_df["Humidity"] < 75) & 
    (cities_df["Cloudiness"] < 30)
]
pref_cities = pref_cities.dropna(how='any')
pref_cities.reset_index(inplace=True)
del pref_cities['index']

# Display to output
pref_cities.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,San Rafael,0,AR,1651962285,28,-34.6177,-68.3301,69.19,12.95
1,Mishawaka,0,US,1651962545,30,41.6620,-86.1586,68.47,10.36
2,Jamestown,0,US,1651962585,35,42.0970,-79.2353,62.53,13.80
3,Sakakah,1,SA,1651962586,24,29.9697,40.2064,62.17,1.59
4,Ingham,0,US,1651962540,37,42.6000,-84.3833,63.93,16.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame(pref_cities)

In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
     "radius": 5000, 
     "keyword":"hotel",
     "key": g_key,
}

In [10]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row["Latitude"]
    long = row["Longitude"]
    city_name = row["City Name"]

    # add keyword to params dict
    params["location"] = f"{lat},{long}"

    # assemble url and make API request
    print(f"Retrieving Closest Hotel for Index# {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index,"Hotel Name"] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("No Hotel Found")
        
    print("---------------")
    
    
print("---End of Search---")

Retrieving Closest Hotel for Index# 0: San Rafael.
Closest hotel in San Rafael is Suter Petit Hotel.
---------------
Retrieving Closest Hotel for Index# 1: Mishawaka.
Closest hotel in Mishawaka is Hyatt Place South Bend/Mishawaka.
---------------
Retrieving Closest Hotel for Index# 2: Jamestown.
Closest hotel in Jamestown is DoubleTree by Hilton Jamestown.
---------------
Retrieving Closest Hotel for Index# 3: Sakakah.
Closest hotel in Sakakah is Raoum Inn Hotel.
---------------
Retrieving Closest Hotel for Index# 4: Ingham.
No Hotel Found
---------------
Retrieving Closest Hotel for Index# 5: Mount Isa.
Closest hotel in Mount Isa is Barkly Hotel.
---------------
Retrieving Closest Hotel for Index# 6: Half Moon Bay.
Closest hotel in Half Moon Bay is Beach House Hotel.
---------------
Retrieving Closest Hotel for Index# 7: Santa Cruz de la Sierra.
Closest hotel in Santa Cruz de la Sierra is Marriott Santa Cruz de la Sierra Hotel.
---------------
Retrieving Closest Hotel for Index# 8: Ci

In [16]:
hotel_df.rename(columns={"City Name" : "City"}, inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,San Rafael,0,AR,1651962285,28,-34.6177,-68.3301,69.19,12.95,Suter Petit Hotel
1,Mishawaka,0,US,1651962545,30,41.6620,-86.1586,68.47,10.36,Hyatt Place South Bend/Mishawaka
2,Jamestown,0,US,1651962585,35,42.0970,-79.2353,62.53,13.80,DoubleTree by Hilton Jamestown
3,Sakakah,1,SA,1651962586,24,29.9697,40.2064,62.17,1.59,Raoum Inn Hotel
4,Ingham,0,US,1651962540,37,42.6000,-84.3833,63.93,16.11,NaN
5,Mount Isa,0,AU,1651962533,48,-20.7333,139.5000,62.37,8.05,Barkly Hotel
6,Half Moon Bay,0,US,1651962304,64,37.4636,-122.4286,69.89,26.46,Beach House Hotel
7,Santa Cruz de la Sierra,20,BO,1651962529,47,-17.8000,-63.1667,78.85,3.44,Marriott Santa Cruz de la Sierra Hotel
8,Cidreira,0,BR,1651962608,74,-30.1811,-50.2056,63.01,8.32,Hotel Castelo
9,Mar del Plata,0,AR,1651962487,74,-38.0023,-57.5575,66.94,11.50,Hotel CR Playa Suites


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [19]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))